In [1]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

df = pd.read_json('datasets/HotPotQA/hotpot_dev_distractor_v1.json')

## Experiment


### Graph creation and refinement section

To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python evalscripts/HotPotQA/hotpot_evaluate_v1.py evalscripts/HotPotQA/sample_dev_pred.json evalscripts/HotPotQA/hotpot_dev_distractor_v1.json
```

In [2]:
context_list = df['context']

In [ ]:
df.head(31)

In [ ]:
df.iloc[7]['context']

In [ ]:
context_list[21:31]

In [ ]:
propositions = []

for i, value in enumerate(tqdm(context_list[21:31])):
        for j, context in enumerate(value):
                index = 1
                if index < len(context):
                    words = context[1]
                    combined = " ".join(words)
                    get_propositions_nosplit(combined, propositions)

In [ ]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

In [ ]:
list_of_edges = generateEdges(propositions_from_file)

In [ ]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

In [ ]:
qg = QueryGraph()

if refine(qg, "person"):
    print("yes")
else:
    print("no")

### Q and A section

In [ ]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [2]:
qg = QueryGraph()
my_dict = {}

# Define a function to apply to each row
def print_qa(row):
    
    question = row['question']
    questionid = row['_id']
    req = qg.get_requirements(question)
    result = qg.answer_question(question, req.content)

    if result is not None:
    
        model_answer = result['result']

        if "don't know the answer" in model_answer:
            model_answer = ""
    
        if len(row['answer']) > 0:
            real_answer = row['answer']
        else:
            real_answer = "\"\""

        if len(result['intermediate_steps']) > 0:
            cypher_query = result['intermediate_steps'][0]['query']
            context = result['intermediate_steps'][1]['context']
       
            if(len(context) < 1):
            #     result = refine_query(cypher_query[6:], question)
            #     model_answer = result['result'] 
                model_answer = "\"\""
                my_dict.update({questionid: ""})
            else:
                my_dict.update({questionid: model_answer})
                
        with open("preds.txt", "a") as preds:
            preds.write("question: " + question + "\n")
            preds.write("real_answer: " + real_answer + "\n")
            preds.write("model_answer: " + model_answer + "\n")  
            preds.write("cypher_query: " + cypher_query + "\n")
            preds.write("======================" + "\n")  
        preds.close()

    else: 
        my_dict.update({questionid: ""})
        

In [3]:
my_dict = {}
# Apply the function to each row
df[0:31].progress_apply(print_qa, axis=1)
    
with open("dev_fullwiki_pred.json", "a") as json_file:
    json.dump(my_dict, json_file, indent=4)

  0%|          | 0/31 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH ()-[r1]-()
WHERE (
    toLower(r1.metadata) =~ '.*\\b(scott derrickson)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(scott derrickson)\\b.*'
)
RETURN DISTINCT r1.metadata AS info1
UNION ALL
MATCH ()-[r2]-()
WHERE (
    toLower(r2.metadata) =~ '.*\\b(ed wood)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(ed wood)\\b.*'
)
RETURN DISTINCT r2.metadata AS info1

Full Context:
[{'info1': '{"summary": "Adam Collis\'s first work was as the assistant director for Scott Derrickson\'s short film \'Love in the Ruins\' in 1995.", "generated_at": "2024-11-07 13:49:30.781924"}'}, {'info1': '{"summary": "Tyler Bates has collaborated with Scott Derrickson.", "generated_at": "2024-11-07 13:49:30.781924"}'}, {'info1': '{"summary": "The film Doctor Strange was directed by Scott Derrickson.", "generated_at": "2024-11-07 13:49:30.781924"}'}, {'info1': '{"summary": "Scott Derrickson wrote the film Doctor Strange with Jon Spaihts 

  6%|▋         | 2/31 [00:07<01:49,  3.78s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(kiss and tell)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(corliss archer)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(government|position)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(government|position)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description

Full Context:
[]


 10%|▉         | 3/31 [00:13<02:14,  4.79s/it]

...

Generated Cypher:
cypher
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(kaiser ventures)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(kaiser ventures)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(american industrialist)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(american industrialist)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description
UNION ALL
MATCH (w)-[r3]-(x)
WHERE (
    toLower(r3.metadata) =~ '.*\\b(father of modern american shipbuilding)\\b.*' OR
    toLower(r3.description) =~ '.*\\b(father of modern american shipbuilding)\\b.*'
)
RETURN DISTINCT r3.metadata AS info1

Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: All sub queries in an UNION must have the same return column names (line 12, column 1 (offset: 385))
"UNION ALL"
 ^}


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\

100%|██████████| 31/31 [03:21<00:00,  6.85s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(poison)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(poison)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(shut up, make love)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(shut up, make love)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description

Full Context:
[{'r1.metadata': '{"summary": "\\"Shut Up, Make Love\\" is performed by the American rock band Poison.", "generated_at": "2024-11-07 19:27:15.009521"}', 'r1.description': "The event 'Shut Up, Make Love' is performed by the object 'Poison'.", 'r2.metadata': '{"summary": "\\"Shut Up, Make Love\\" is a song.", "generated_at": "2024-11-07 19:27:15.009521"}', 'r2.description': "The entity 'Shut Up, Make Love' is categorized as a song, which is an event."}, {'r1.metadata': '{"summary": "\\"Shut Up, Make Love\\" was the first Poison single with 

100%|██████████| 31/31 [03:26<00:00,  6.67s/it]


> Finished chain.


In [ ]:
context_list[18]

In [ ]:
x = df.iloc[31]
question = x['question']
print("question:", question)
answer = x['answer']
print("answer:", answer)

qg = QueryGraph()
req = qg.get_requirements(question)
res = qg.answer_question(question, req.content)
res

## Graph refinement 

In [ ]:
qg = QueryGraph()

QUERY = """
MATCH (N) RETURN N
"""
res = qg._graph.query(QUERY)
res

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [ ]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

Create embeddings

In [ ]:
# # Now we initialize from existing graph
# existing_graph = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
#     url=neo4j_url,
#     username=neo4j_user,
#     password=neo4j_password,
#     index_name="person_index",
#     node_label="Person",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

Refactor node labels

In [ ]:
# # cq = """
# # MATCH (en:Entity)
# # WHERE en.label = 'Person'
# # WITH collect(en) AS persons
# # CALL apoc.refactor.rename.label("Entity", "Person", persons)
# # YIELD batches, total, timeTaken, committedOperations
# # RETURN batches, total, timeTaken, committedOperations;
# # """
# # res = qg._graph.query(cq)

Create projection

In [ ]:
cq = """
MATCH (p:Person)
RETURN gds.graph.project(
  'personproj',
  p,
  null,
  {
    sourceNodeProperties: p { .embedding },
    targetNodeProperties: {}
  }
)
"""

res = qg._graph.query(cq)

Use KNN to measure node similarities

In [ ]:
cq = """
CALL gds.knn.stream('projection2', {
topK: 1,
nodeProperties: ['embedding'],
// The following parameters are set to produce a deterministic result
randomSeed: 1337,
concurrency: 1,
sampleRate: 1.0,
deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE similarity > .90
RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

res = qg._graph.query(cq)
res

In [ ]:
for i in res:
    for key, value in i.items():
        print(f"{key}: {value}")

In [ ]:
def are_dictionaries_equivalent(d1, d2):
    # Check if both have the same keys
    if d1.keys() != d2.keys():
        return False

    # Check if Person1 and Person2 are swapped but equivalent
    if (d1['Person1'] == d2['Person2'] and d1['Person2'] == d2['Person1'] and d1['similarity'] == d2['similarity']):
        return True
    
    return False


In [ ]:
newlist = res

In [ ]:
for idx, val in enumerate(res):   
    for idx2, val2 in enumerate(newlist):
        if val != val2:
            if are_dictionaries_equivalent(val, val2):
                newlist.pop(idx2)


In [ ]:
unique = []

for value in newlist:
    if value not in unique:
        unique.append(value)

unique

In [ ]:
node_name = "Shirley Temple"

query = f"""
MATCH (n:Person {{name: '{node_name}'}})
RETURN n
"""
print(query)

res = qg._graph.query(query)
res


Merge similar nodes (Entity resolution)

In [ ]:
for idx, val in enumerate(unique):
    print(val)
    p1 = val['Person1']
    p2 = val['Person2']    
    cq = f"""
    MATCH (a1:Person {{name: '{p1}'}}), (a2:Person {{name: '{p2}'}})
    WITH head(collect([a1,a2])) as nodes
    CALL apoc.refactor.mergeNodes(nodes,{{
    properties:"discard",
    mergeRels:true
    }})
    YIELD node
    RETURN node;
    """
    res = qg._graph.query(cq)
    res